In [2]:
from tools import get_puzzle, show_problem_1, show_problem_2

TODAY = 8
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)

https://adventofcode.com/2023/day/8
## --- Day 8: Haunted Wasteland ---


You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about **ghosts** a few minutes ago.


One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of **network** of labeled nodes.


It seems like you're meant to use the **left/right** instructions to **navigate the network** . Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!


After examining the maps for a bit, two nodes stick out:AAA``andZZZ``. You feel likeAAA``is where you are now, and you have to follow the left/right instructions until you reachZZZ``.


This format defines each **node** of the network individually. For example:


```
 RL
 
 AAA = (BBB, CCC)
 BBB = (DDD, EEE)
 CCC = (ZZZ, GGG)
 DDD = (DDD, DDD)
 EEE = (EEE, EEE)
 GGG = (GGG, GGG)
 ZZZ = (ZZZ, ZZZ)

```


Starting withAAA``, you need to **look up the next element** based on the next left/right instruction in your input. In this example, start withAAA``and go **right** (R``) by choosing the right element ofAAA``,CCC``. Then,L``means to choose the **left** element ofCCC``,ZZZ``. By following the left/right instructions, you reachZZZ``in2``steps.


Of course, you might not findZZZ``right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary:RL``really meansRLRLRLRLRLRLRLRL...``and so on. For example, here is a situation that takes6``steps to reachZZZ``:


```
 LLR
 
 AAA = (BBB, BBB)
 BBB = (AAA, ZZZ)
 ZZZ = (ZZZ, ZZZ)

```


Starting atAAA``, follow the left/right instructions. **How many steps are required to reachZZZ``?** 




In [3]:
from itertools import cycle

class Node():
    def __init__(self, line):
        self.name = line[0:3]
        self.l = line[7:10]
        self.r = line[12:15]
    
    def step(self,step):
        assert step in ["L", "R"]
        if step == "L":
            return self.l
        return self.r

    def __repr__(self):
        return f"{self.name} ({self.l},{self.r})"

def parse_data(data):
    step_it = data[0]

    nodes = {}
    for line in data [2:]:
        node = Node(line)
        nodes[node.name] = node
    return Stepper(step_it), nodes

class Stepper():
    def __init__(self,steps_str):
        self.steps_str = steps_str
        self._it = cycle(self.steps_str)
    
    def next(self):
        return next(self._it)

    
def solution_1(data):
    stepper, nodes = parse_data(data)

    stop = False
    curr_name = 'AAA'
    curr_node = nodes[curr_name]
    count = 0

    while not stop:
        step = stepper.next()
        count += 1
        curr_node = nodes[curr_node.step(step)]
        #print(f"{step} -> {curr_node}")
        if curr_node.name == 'ZZZ':
            stop = True
    return count


assert solution_1 (puzzle.test) == 2

print( f"Solution 1:  {solution_1 (puzzle.data)} are the steps are required to reach ZZZ")  # 14681 0.1s

Solution 1:  14681 are the steps are required to reach ZZZ


In [3]:
show_problem_2(puzzle)

## --- Part Two ---


Thesandstormis upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take **significantly more steps** to escape!


What if the map isn't for people - what if the map is for **ghosts** ? Are ghosts even bound by the laws of spacetime? Only one way to find out.


After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending inA``is equal to the number ending inZ``! If you were a ghost, you'd probably just **start at every node that ends withA``** and follow all of the paths at the same time until they all simultaneously end up at nodes that end withZ``.


For example:


```
 LR
 
 11A = (11B, XXX)
 11B = (XXX, 11Z)
 11Z = (11B, XXX)
 22A = (22B, XXX)
 22B = (22C, 22C)
 22C = (22Z, 22Z)
 22Z = (22B, 22B)
 XXX = (XXX, XXX)

```


Here, there are two starting nodes,11A``and22A``(because they both end withA``). As you follow each left/right instruction, use that instruction to **simultaneously** navigate away from both nodes you're currently on. Repeat this process until **all** of the nodes you're currently on end withZ``. (If only some of the nodes you're on end withZ``, they act like any other node and you continue as normal.) In this example, you would proceed as follows:


+ Step 0: You are at11A``and22A``.
+ Step 1: You choose all of the **left** paths, leading you to11B``and22B``.
+ Step 2: You choose all of the **right** paths, leading you to11Z``and22C``.
+ Step 3: You choose all of the **left** paths, leading you to11B``and22Z``.
+ Step 4: You choose all of the **right** paths, leading you to11Z``and22B``.
+ Step 5: You choose all of the **left** paths, leading you to11B``and22C``.
+ Step 6: You choose all of the **right** paths, leading you to11Z``and22Z``.


So, in this example, you end up entirely on nodes that end inZ``after6``steps.


Simultaneously start on every node that ends withA``. **How many steps does it take before you're only on nodes that end withZ``?** 




In [30]:
from functools import reduce

DEBUG = False

test2 = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)""".splitlines()

class Node():
    def __init__(self, line):
        self.name = line[0:3]
        self.l = line[7:10]
        self.r = line[12:15]
        self.periods = []
    
    def step(self,step):
        assert step in ["L", "R"]
        if step == "L":
            return self.l
        return self.r
    
    def min_period (self):
        return min([b-a for a,b in zip(self.periods, self.periods[1:])]) 

    def __repr__(self):
        return f"{self.name} ({self.l},{self.r}) {self.periods}"

def solution_2(data):
    stepper, nodes = parse_data(data)

   
    a_nodes = [nodes[n.name] for n in nodes.values() if n.name[2] == 'A']
    curr_nodes = [n for n in a_nodes]
    
    step = stepper.next()

    curr_nodes = [nodes[a.step(step)] for a in a_nodes]
    orig_nodes = [node for node in curr_nodes]
    orig_names = [node.name for node in curr_nodes ]

    print(f"starting nodes = {curr_nodes}")
    print(f"orig_names = {orig_names}")


    count = 0
    while count < 8000:
        step = stepper.next()
        count += 1
        curr_nodes = [nodes[curr_node.step(step)] for curr_node in curr_nodes]
      
        curr_names = [curr_node.name for curr_node in curr_nodes]
        for curr, orig in zip(curr_names, orig_names):
            if curr == orig:
                found = nodes[curr]
                found.periods.append(count)
                if DEBUG: print(f" adding a period for {found.name} to {count}")
                if DEBUG: print(f" they now are: {found.periods}")
                break

                
                
    
    #orig_periods = [orig.periods for orig in orig_nodes]


    #We calculate the 'distance counts' betweeen every occurrence of a **Z by
    minimums = [ orig.min_period() for orig in orig_nodes]

    print (f"minimums are {minimums}")


    # we now multiply all of those times the length of the pattern

    return reduce(lambda a,b: a*b, minimums)* len (data[0])

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

#assert solution_2 (test2) == 6  This is not working because we need to make sure some numbers are not multiple of others

print( f"Solution 2:  {solution_2 (puzzle.data)} are the steps are required to reach **Z on every starting node (ending with A) ")  # 14321394058031 0.1s

starting nodes = [JPD (KKV,FQP) [], NHL (LDP,RNH) [], FVR (LXP,CDN) [], NFG (DSR,HNX) [], LQS (TCN,FLS) [], KRR (LXS,KBF) []]
orig_names = ['JPD', 'NHL', 'FVR', 'NFG', 'LQS', 'KRR']
minimums are [61, 59, 79, 47, 53, 73]
Solution 2:  14321394058031 are the steps are required to reach **Z on every starting node (ending with A) 
